In [3]:
!pip install pymorphy2

  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)


# Загрузка датасета и определение функций

In [1]:
import pymorphy2
import pandas as pd
import re

In [8]:
df = pd.read_excel('C:/Users/vchemsmisl/Desktop/стафф/стажировка/Результаты ВБ_афазия.xlsx')

In [9]:
df

,Unnamed: 0,ID,Age,Aphasia type,Severity,этиол,диагноз клин,Education years,Gender,Handedness,Native language,C6(a),C6(a) - words (N),C6(b),C6(b) - words (N),C6(c),C6(c) - words (N)
0,0,P001,32.008219,c_mot,mild,stroke,"Последствия субарахноидального кровоизлияния, ...",15.0,f,right,Russian,"животных, да? сколько можно, да? всех не переч...",NaN,"врач, э пожарный, э, милиционер, так, м, всех ...",NaN,"Москва, Сибирь, Омск, Новокузнецк, э, Париж, А...",NaN
1,1,P003,56.131507,"ef_mot, aff_mot",mod,stroke,"Последствия внутричерепного кровоизлияния, Пос...",13.0,m,right,Russian,"корова, поросенок, овца, слон, лев, тигр",NaN,"директор, сантехник, учитель",NaN,"Москва, Вологда, Волгоград, Сочи, Анапа, Алушта",NaN
2,2,P004,59.967123,ef_mot,mild,stroke,"Последствия инфаркта мозга, в бассейне левой с...",13.0,f,right,Russian,"так, всех за одну минуту? заяц, я- лиса, волк,...",NaN,"слесарь, плотик, сантехник, телефонист, инжене...",NaN,"города? Москва, Казань, Саратов, Воронеж, Тула...",NaN
3,3,P006,34.578082,c_mot,mild-mod,stroke,"\nПоследствия инфаркта мозга, в бассей...",15.0,m,right,Russian,"так, лиса, волк, э, лев, э, тигр, гепард, леоп...",NaN,"э, технолог, банкир, м, так, э, даже не знаю, ...",NaN,"Москва, Санкт-Петербург, ма-, Великий Новгород...",NaN
4,4,P008,64.389041,"c_mot (af-ef), dyn",mod,stroke,"Последствия инфаркта мозга, Последствия инфарк...",9.0,m,right,Russian,"животные. я этот собака, кошка, ну все. свинья...",NaN,"профессии? ну, плотник, столяр, бетонщик, мед-...",NaN,"города. Москва, Бергород, берго-берго-берго-, ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,63,P096,57.994521,dys,mod,stroke,"\t\nПоследствия инфаркта мозга, Повторный инфа...",13.5,m,right,Russian,"корова, лошадь, обезьяна, животные, крокодил, ...",13.0,"водитель, сантехник, электрик, токарь, швея, п...",19.0,"Москва, Ленинград (х2), Свердловск, Владивосто...",23.0
64,64,P097,63.536986,dys,mod,stroke,"\t\nПоследствия инфаркта мозга, Поздний восста...",17.0,m,right,Russian,"животные... животные.. кошка, собака, пони, ло...",6.0,"столяр, маляр, кочегар, плотник, директор, да,...",6.0,"Москва, Днепропетровск, Кострома, Лужково, Пет...",9.0
65,65,P101,69.791781,dys,mild,stroke,"\t\nПоследствия инфаркта мозга, Состояние посл...",17.0,f,right,Russian,"животные.. слон, бык, теленок, свинья, поросен...",17.0,"учитель, повар, инструктор, шофер, врач, летчи...",15.0,"Клин, Калинин, Москва, Крюково, Зеленоград, Со...",20.0
66,66,P104,71.794521,dys,mild,stroke,"Последствия инфаркта мозга, Ранний восстановит...",17.0,m,right,Russian,"обезьяна, кит, жираф, лев, кобыла, як, корова,...",13.0,"сварщик, инженер, космонавт, член правительств...",14.0,"Рио-де-Ж, Москва, Париж, Киншаса, Антверпен, Д...",17.0


In [10]:
class TextProcessor:
    
    def __init__(
        self, 
        class_tag: str, 
        dataframe: pd.DataFrame,
        stop_words: list) -> None:
        
        self._class_tag = class_tag
        self._stop_words = stop_words
        self._df = dataframe
        self._parser = pymorphy2.MorphAnalyzer()
        
    def _tokenize_string(self, words_string: str) -> list:
        words_string = words_string.lower()
        words_string = re.sub(r'\w+\?', '', words_string)
        words_string = re.sub(r'[\?.:;!,]', ' ', words_string)
        words_string = re.sub(r'[(\*].+?[)\*]', '', words_string)
        words_string = re.sub(r'(?<=нижний)\s+', '_', words_string)
        words_string = re.sub(r'(?<=великий)\s+', '_', words_string)
        
        token_list = words_string.split(' ')
        token_list = [token for token in token_list if token]
    
        return token_list
    
    def _lemmatize_and_sort_out_irrelevant_words(self, token_list: list) -> tuple:
        rel_token_list = []
        irrel_token_list = []
        
        for token in token_list:
            open_corp_tags = self._parser.tag(token)
            normal_forms = self._parser.normal_forms(token)
                
            if open_corp_tags[0].POS == 'NOUN' and \
            len(normal_forms[0]) > 1 and \
            normal_forms[0] not in self._stop_words:
                if normal_forms[0] == 'белок': # решения изящнее не нашёл :)
                    normal_forms[0] = 'белка'
                if normal_forms[0] in rel_token_list:
                    continue
                rel_token_list.append(normal_forms[0])
            
            # для слов типа "столяр"
            elif len(open_corp_tags) > 1 and open_corp_tags[1].POS == 'NOUN':
                if len(normal_forms) > 1 and \
                len(normal_forms[1]) > 1 and \
                normal_forms[1] not in self._stop_words:
                    if normal_forms[1] == 'белок': 
                        normal_forms[1] = 'белка'
                    if normal_forms[1] in rel_token_list:
                        continue
                    rel_token_list.append(normal_forms[1])
                    
                elif len(normal_forms[0]) > 1 and normal_forms[0] not in self._stop_words:
                    if normal_forms[0] == 'белок':
                        normal_forms[0] = 'белка'
                    if normal_forms[0] in rel_token_list:
                        continue
                    rel_token_list.append(normal_forms[0])
                else:
                    irrel_token_list.append(token)
            
            # для слов типа "военный"
            elif len(open_corp_tags) > 2 and open_corp_tags[2].POS == 'NOUN':
                if len(normal_forms) > 1 and \
                len(normal_forms[1]) > 1 and \
                normal_forms[1] not in self._stop_words:
                    if normal_forms[1] == 'белок': 
                        normal_forms[1] = 'белка'
                    if normal_forms[1] in rel_token_list:
                        continue
                    rel_token_list.append(normal_forms[1])
                    
                elif len(normal_forms[0]) > 1 and normal_forms[0] not in self._stop_words:
                    if normal_forms[0] == 'белок':
                        normal_forms[0] = 'белка'
                    if normal_forms[0] in rel_token_list:
                        continue
                    rel_token_list.append(normal_forms[0])
                else:
                    irrel_token_list.append(token)   
                    
            else:
                irrel_token_list.append(token)

        return rel_token_list, irrel_token_list
    
    def _write_to_dataset(self, tokens_list: list) -> None:
        self._df[f'{self._class_tag}-clean'] = tokens_list
        
    def process_strings(self) -> list:
        words_strings = []
        for string in self._df[self._class_tag]:
        
            token_list = self._tokenize_string(string)
            words, pauses_fillers = self._lemmatize_and_sort_out_irrelevant_words(token_list)
            words_strings.append(', '.join(words))
        self._write_to_dataset(words_strings)
            
        return pauses_fillers

In [5]:
titles = ['C6(a)', 'C6(b)', 'C6(c)']
# стоп-слова, полученные эмпирически
stop_words = ['господь', 'пот', 'потом', 'да', 'слух',
             'ответ', 'категория', 'нерв', 'блин', 'эм', 
             'че', 'чета', 'тута']

# Тестирую на части данных

In [24]:
temp_df = df.loc[:20]
temp_df

,Unnamed: 0,ID,Age,Aphasia type,Severity,этиол,диагноз клин,Education years,Gender,Handedness,Native language,C6(a),C6(a) - words (N),C6(b),C6(b) - words (N),C6(c),C6(c) - words (N)
0,0,P001,32.008219,c_mot,mild,stroke,"Последствия субарахноидального кровоизлияния, ...",15.0,f,right,Russian,"животных, да? сколько можно, да? всех не переч...",NaN,"врач, э пожарный, э, милиционер, так, м, всех ...",NaN,"Москва, Сибирь, Омск, Новокузнецк, э, Париж, А...",NaN
1,1,P003,56.131507,"ef_mot, aff_mot",mod,stroke,"Последствия внутричерепного кровоизлияния, Пос...",13.0,m,right,Russian,"корова, поросенок, овца, слон, лев, тигр",NaN,"директор, сантехник, учитель",NaN,"Москва, Вологда, Волгоград, Сочи, Анапа, Алушта",NaN
2,2,P004,59.967123,ef_mot,mild,stroke,"Последствия инфаркта мозга, в бассейне левой с...",13.0,f,right,Russian,"так, всех за одну минуту? заяц, я- лиса, волк,...",NaN,"слесарь, плотик, сантехник, телефонист, инжене...",NaN,"города? Москва, Казань, Саратов, Воронеж, Тула...",NaN
3,3,P006,34.578082,c_mot,mild-mod,stroke,"\nПоследствия инфаркта мозга, в бассей...",15.0,m,right,Russian,"так, лиса, волк, э, лев, э, тигр, гепард, леоп...",NaN,"э, технолог, банкир, м, так, э, даже не знаю, ...",NaN,"Москва, Санкт-Петербург, ма-, Великий Новгород...",NaN
4,4,P008,64.389041,"c_mot (af-ef), dyn",mod,stroke,"Последствия инфаркта мозга, Последствия инфарк...",9.0,m,right,Russian,"животные. я этот собака, кошка, ну все. свинья...",NaN,"профессии? ну, плотник, столяр, бетонщик, мед-...",NaN,"города. Москва, Бергород, берго-берго-берго-, ...",NaN
5,5,P010,42.328767,ef_mot,mild,stroke,"Последствия инфаркта мозга, Последствия после ...",13.0,m,right,Russian,"кошка, собака, носорог, жираф, волк, медведь, ...",NaN,"учитель, э врач, э, товаровед, продавец, уборщ...",NaN,"Москва, Санкт-Петербург, э Новороссийск, Анапа...",NaN
6,6,P012,68.863014,sens,severe,stroke,"\t\nПоследствия инфаркта мозга, состояние посл...",15.0,m,right,Russian,"ну да много всяких. и фряды, нефти, и кто еще?...",NaN,"спеть, спол, убежал, маро(неразборчиво). вот, ...",NaN,"ортения? ну это, с, это, плавать в чем-то. ну,...",NaN
7,7,P016,69.334247,"ef_mot, af_mot",mild,stroke,"\t\nПоследствия инфаркта мозга, в бассейне лев...",15.0,m,right,Russian,"кошка, собака, э, хрюшка, лошадь, коза, а, вер...",NaN,"профессии. какую? я помню, что я военный. воен...",NaN,"города. Ярославль, как в фильме: я там родился...",NaN
8,8,P017,26.183562,c_mot (af-ef),mod,stroke,"\nПоследствия инфаркта мозга, в бассей...",12.0,f,right,Russian,"э, животные. собака, кошка, петух, э, ну, кури...",NaN,"э, учитель, ну, биснесмен, профессор, лодочник...",NaN,"э, Москва, Санкт-Петербург, Ростов, Нальчик, э...",NaN
9,9,P018,66.635616,sens,mod-severe,stroke,"Последствия инфаркта мозга, с геморрагическим ...",16.0,m,right,Russian,"животных-то может быть. ну, например, кот, коз...",NaN,"профессии, ну, например, ученый, например даже...",NaN,"Москва, с, раньше, Питер, потом Киев, Минск, О...",NaN


In [32]:
all_pauses_fillers = []
for title in titles:
    processor = TextProcessor(title, temp_df, stop_words)
    test_pauses_fillers = processor.process_strings()
    all_pauses_fillers.extend(test_pauses_fillers)


C:\Users\vchemsmisl\AppData\Local\Temp\ipykernel_7452\3460461142.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._df[f'{self._class_tag}-clean'] = tokens_list


In [28]:
temp_df

,Unnamed: 0,ID,Age,Aphasia type,Severity,этиол,диагноз клин,Education years,Gender,Handedness,Native language,C6(a),C6(a) - words (N),C6(b),C6(b) - words (N),C6(c),C6(c) - words (N),C6(a)-clean,C6(b)-clean,C6(c)-clean
0,0,P001,32.008219,c_mot,mild,stroke,"Последствия субарахноидального кровоизлияния, ...",15.0,f,right,Russian,"животных, да? сколько можно, да? всех не переч...",NaN,"врач, э пожарный, э, милиционер, так, м, всех ...",NaN,"Москва, Сибирь, Омск, Новокузнецк, э, Париж, А...",NaN,"щенок, первое, телёнок, овца, животное, корова...","слесарь, психолог, пожарный, сантехник, милици...","киселёв, новокузнецк, америка, омск, сибирь, с..."
1,1,P003,56.131507,"ef_mot, aff_mot",mod,stroke,"Последствия внутричерепного кровоизлияния, Пос...",13.0,m,right,Russian,"корова, поросенок, овца, слон, лев, тигр",NaN,"директор, сантехник, учитель",NaN,"Москва, Вологда, Волгоград, Сочи, Анапа, Алушта",NaN,"лев, слон, овца, поросёнок, корова, тигр","директор, учитель, сантехник","сочи, алушта, волгоград, вологда, москва, анапа"
2,2,P004,59.967123,ef_mot,mild,stroke,"Последствия инфаркта мозга, в бассейне левой с...",13.0,f,right,Russian,"так, всех за одну минуту? заяц, я- лиса, волк,...",NaN,"слесарь, плотик, сантехник, телефонист, инжене...",NaN,"города? Москва, Казань, Саратов, Воронеж, Тула...",NaN,"куница, лось, корова, коза, лиса, госпадь, кро...","телефонист, литейщик, сантехник, почтальон, ин...","тула, саратов, углич, белгород, варшава, ворон..."
3,3,P006,34.578082,c_mot,mild-mod,stroke,"\nПоследствия инфаркта мозга, в бассей...",15.0,m,right,Russian,"так, лиса, волк, э, лев, э, тигр, гепард, леоп...",NaN,"э, технолог, банкир, м, так, э, даже не знаю, ...",NaN,"Москва, Санкт-Петербург, ма-, Великий Новгород...",NaN,"лев, гепард, медведь, ворона, леопард, жираф, ...","банкир, банк, бант, водитель, токарь, все-все-...","пенза, новосибирск, сочи, нижний_норговорода, ..."
4,4,P008,64.389041,"c_mot (af-ef), dyn",mod,stroke,"Последствия инфаркта мозга, Последствия инфарк...",9.0,m,right,Russian,"животные. я этот собака, кошка, ну все. свинья...",NaN,"профессии? ну, плотник, столяр, бетонщик, мед-...",NaN,"города. Москва, Бергород, берго-берго-берго-, ...",NaN,"обезьяна, животное, собака, кошка, свинья","бетонщик, врач, плотник, столяр","город, бергогод, москва, бергород"
5,5,P010,42.328767,ef_mot,mild,stroke,"Последствия инфаркта мозга, Последствия после ...",13.0,m,right,Russian,"кошка, собака, носорог, жираф, волк, медведь, ...",NaN,"учитель, э врач, э, товаровед, продавец, уборщ...",NaN,"Москва, Санкт-Петербург, э Новороссийск, Анапа...",NaN,"носорог, медведь, ворона, леопард, слон, жираф...","продавец, сиделка, психолог, машинист, по-в, д...","калуга, боровск, страна, москва, анапа, малояр..."
6,6,P012,68.863014,sens,severe,stroke,"\t\nПоследствия инфаркта мозга, состояние посл...",15.0,m,right,Russian,"ну да много всяких. и фряды, нефти, и кто еще?...",NaN,"спеть, спол, убежал, маро(неразборчиво). вот, ...",NaN,"ортения? ну это, с, это, плавать в чем-то. ну,...",NaN,"голова, саляб, фряд, зыбка, нефть, шибел, мышк...","дырка, ситони, стантея, спол, жалбище, маро","америка, иза, фотография, москва, стольдиния, ..."
7,7,P016,69.334247,"ef_mot, af_mot",mild,stroke,"\t\nПоследствия инфаркта мозга, в бассейне лев...",15.0,m,right,Russian,"кошка, собака, э, хрюшка, лошадь, коза, а, вер...",NaN,"профессии. какую? я помню, что я военный. воен...",NaN,"города. Ярославль, как в фильме: я там родился...",NaN,"медведь, верблюд, животное, лошадь, коза, хрюш...","студент, скатя, разновидность, строитель, рабо...","житомир, днепропетровск, переславль, ростов, в..."
8,8,P017,26.183562,c_mot (af-ef),mod,stroke,"\nПоследствия инфаркта мозга, в бассей...",12.0,f,right,Russian,"э, животные. собака, кошка, петух, э, ну, кури...",NaN,"э, учитель, ну, биснесмен, профессор, лодочник...",NaN,"э, Москва, Санкт-Петербург, Ростов, Нальчик, э...",NaN,"гусь, ти, курица, медведь, рысь, животное, эма...","депутат, рабочий, учитель, профессор, коробль,...","новоси

In [29]:
all_pauses_fillers = set([token for token in all_pauses_fillers if token])
all_pauses_fillers

{'гикая', 'домашняя', 'ой', 'р'}

# Теперь полная обработка

In [11]:
titles = ['C6(a)', 'C6(b)', 'C6(c)']
stop_words = ['господь', 'пот', 'потом', 'да', 'животное', 'профессия', 'город', 'слух',
             'ответ', 'категория']

all_pauses_fillers = []
for title in titles:
    processor = TextProcessor(title, df, stop_words)
    pauses_fillers = processor.process_strings()
    print(pauses_fillers)
    all_pauses_fillers.extend(pauses_fillers)

['там']
[]
[]


In [12]:
df

,Unnamed: 0,ID,Age,Aphasia type,Severity,этиол,диагноз клин,Education years,Gender,Handedness,Native language,C6(a),C6(a) - words (N),C6(b),C6(b) - words (N),C6(c),C6(c) - words (N),C6(a)-clean,C6(b)-clean,C6(c)-clean
0,0,P001,32.008219,c_mot,mild,stroke,"Последствия субарахноидального кровоизлияния, ...",15.0,f,right,Russian,"животных, да? сколько можно, да? всех не переч...",NaN,"врач, э пожарный, э, милиционер, так, м, всех ...",NaN,"Москва, Сибирь, Омск, Новокузнецк, э, Париж, А...",NaN,"животный, кошка, собака, тигр, щенок, корова, ...","врач, пожарный, милиционер, го-голова, психоло...","москва, сибирь, омск, новокузнецк, париж, амер..."
1,1,P003,56.131507,"ef_mot, aff_mot",mod,stroke,"Последствия внутричерепного кровоизлияния, Пос...",13.0,m,right,Russian,"корова, поросенок, овца, слон, лев, тигр",NaN,"директор, сантехник, учитель",NaN,"Москва, Вологда, Волгоград, Сочи, Анапа, Алушта",NaN,"корова, поросёнок, овца, слон, лев, тигр","директор, сантехник, учитель","москва, вологда, волгоград, сочи, анапа, алушта"
2,2,P004,59.967123,ef_mot,mild,stroke,"Последствия инфаркта мозга, в бассейне левой с...",13.0,f,right,Russian,"так, всех за одну минуту? заяц, я- лиса, волк,...",NaN,"слесарь, плотик, сантехник, телефонист, инжене...",NaN,"города? Москва, Казань, Саратов, Воронеж, Тула...",NaN,"заяц, лиса, волк, жираф, слон, куница, крокоди...","слесарь, плотик, сантехник, телефонист, инжене...","москва, казань, саратов, воронеж, тула, алмат,..."
3,3,P006,34.578082,c_mot,mild-mod,stroke,"\nПоследствия инфаркта мозга, в бассей...",15.0,m,right,Russian,"так, лиса, волк, э, лев, э, тигр, гепард, леоп...",NaN,"э, технолог, банкир, м, так, э, даже не знаю, ...",NaN,"Москва, Санкт-Петербург, ма-, Великий Новгород...",NaN,"лиса, волк, лев, тигр, гепард, леопард, медвед...","технолог, банкир, банк, все-все-все, бант, муз...","москва, санкт-петербург, великий_новгород, ниж..."
4,4,P008,64.389041,"c_mot (af-ef), dyn",mod,stroke,"Последствия инфаркта мозга, Последствия инфарк...",9.0,m,right,Russian,"животные. я этот собака, кошка, ну все. свинья...",NaN,"профессии? ну, плотник, столяр, бетонщик, мед-...",NaN,"города. Москва, Бергород, берго-берго-берго-, ...",NaN,"собака, кошка, свинья, обезьяна","плотник, столяр, бетонщик, врач","москва, бергород, бергогод"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,63,P096,57.994521,dys,mod,stroke,"\t\nПоследствия инфаркта мозга, Повторный инфа...",13.5,m,right,Russian,"корова, лошадь, обезьяна, животные, крокодил, ...",13.0,"водитель, сантехник, электрик, токарь, швея, п...",19.0,"Москва, Ленинград (х2), Свердловск, Владивосто...",23.0,"корова, лошадь, обезьяна, крокодил, жираф, ум,...","водитель, сантехник, электрик, токарь, швея, п...","москва, ленинград, свердловск, владивосток, пе..."
64,64,P097,63.536986,dys,mod,stroke,"\t\nПоследствия инфаркта мозга, Поздний восста...",17.0,m,right,Russian,"животные... животные.. кошка, собака, пони, ло...",6.0,"столяр, маляр, кочегар, плотник, директор, да,...",6.0,"Москва, Днепропетровск, Кострома, Лужково, Пет...",9.0,"кошка, собака, пони, лошадь, журавль, конь","столяр, маляр, кочегар, плотник, директор, доктор","москва, днепропетровск, кострома, петербург, с..."
65,65,P101,69.791781,dys,mild,stroke,"\t\nПоследствия инфаркта мозга, Состояние посл...",17.0,f,right,Russian,"животные.. слон, бык, теленок, свинья, поросен...",17.0,"учитель, повар, инструктор, шофер, врач, летчи...",15.0,"Клин, Калинин, Москва, Крюково, Зеленоград, Со...",20.0,"слон, бык, телёнок, свинья, поросёнок, кенгур,...","учитель, повар, инструктор, шофер, врач, лётчи...","клин, калинин, москва, крюково, зеленоград, со..."
66,66,P104,71.794521,dys,mild,stroke,"Последствия инфаркта мозга, Ранний восстановит...",17.0,m,right,Russian,"обезьяна, кит, жираф, лев, кобыла, як, корова,...",13.0,"сварщик, инженер, космонавт, член правительств...",14.0,"Рио-де-Ж, Москва, Париж, Киншаса, Антверпен, Д...",17.0,"обезьяна, кит, жираф, лев, кобыла, як, корова,...

In [14]:
all_pauses_fillers = set([token for token in all_pauses_fillers if token])
all_pauses_fillers

{'там'}

In [13]:
df.to_excel("C:/Users/vchemsmisl/Desktop/стафф/стажировка/Результаты ВБ_афазия_cleaned.xlsx", sheet_name='aphasia', na_rep='NA')